In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
from collections import Counter
import re

In [3]:
# Fonction pour charger les données
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if '\t' in line:
                text, label = line.strip().split('\t')
                data.append((text, label))
    return data

# Testons la fonction
train_data = load_data('../data/train.txt')
print(f"Nombre de tweets chargés: {len(train_data)}")
print("Exemples des 3 premiers tweets:")
for i in range(min(3, len(train_data))):
    print(f"Tweet: {train_data[i][0]}")
    print(f"Étiquette: {train_data[i][1]}")
    print()

Nombre de tweets chargés: 7136
Exemples des 3 premiers tweets:
Tweet: Saint-Nazaire se rêve en capitale des #énergies marines renouvelables #electricite http://t.co/nArcLYqxLd
Étiquette: =

Tweet: 4eme Conférence internationale sur le changement climatique et le développement http://t.co/biVoGZVlLj
Étiquette: =

Tweet: Rencontres #windustry 2014 Sascha Wiesner décrit les innovations d'A2SEA dans le transport des éoliennes offshore http://t.co/hPD0DQPay6
Étiquette: =



In [4]:
train_df = pd.DataFrame(train_data, columns=['text', 'label'])

print(train_df.head())

                                                text label
0  Saint-Nazaire se rêve en capitale des #énergie...     =
1  4eme Conférence internationale sur le changeme...     =
2  Rencontres #windustry 2014 Sascha Wiesner décr...     =
3  #Photos :Dans l’Ouest américain,les stigmates ...     -
4  Parc #éolien: entente conclut entre Port-Carti...     +


In [5]:
# Distribution des étiquettes
label_counts = train_df['label'].value_counts()
print("Distribution des étiquettes:")
print(label_counts)

# Pourcentage
label_percentages = 100 * label_counts / len(train_df)
print("\nPourcentage par étiquette:")
print(label_percentages.round(2))

# Exemples de tweets pour chaque catégorie
print("\nExemples de tweets par catégorie:")
for label in ['+', '-', '=']:
    print(f"\nCatégorie: {label}")
    examples = train_df[train_df['label'] == label]['text'].head(2).values
    for ex in examples:
        print(f"- {ex}")

Distribution des étiquettes:
label
=    3205
+    2234
-    1697
Name: count, dtype: int64

Pourcentage par étiquette:
label
=    44.91
+    31.31
-    23.78
Name: count, dtype: float64

Exemples de tweets par catégorie:

Catégorie: +
- Parc #éolien: entente conclut entre Port-Cartier la compagnie Système d'énergie renouvelable Canada: http://t.co/G9DWR1T792 #CoteNord
- Frédéric Chauvel : "attirer les investisseurs en créant un écosystème favorisant" #AFE2014

Catégorie: -
- #Photos :Dans l’Ouest américain,les stigmates d’une sécheresse historique http://t.co/F6As9walnX #Environnement #Climat #EtatsUnis #Ecologie
- Oublié le tps où Le Drian écrivait au ministre de la défense se plaigant de l’armée empêchant dvpt de l’éolien en Bzh http://t.co/JTlx2ngbe4

Catégorie: =
- Saint-Nazaire se rêve en capitale des #énergies marines renouvelables #electricite http://t.co/nArcLYqxLd
- 4eme Conférence internationale sur le changement climatique et le développement http://t.co/biVoGZVlLj


In [6]:
# Fonction de prétraitement basique
def preprocess_text(text):
    # Conversion en minuscules
    text = text.lower()
    # Suppression de la ponctuation et caractères spéciaux
    text = re.sub(r'[^\w\s]', ' ', text)
    # Suppression des espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Testons le prétraitement sur quelques exemples
print("Exemples de prétraitement:")
for i in range(3):
    original = train_df['text'].iloc[i]
    processed = preprocess_text(original)
    print(f"Original: {original}")
    print(f"Prétraité: {processed}")
    print()

# Ajoutons une colonne avec les textes prétraités
train_df['processed_text'] = train_df['text'].apply(preprocess_text)

Exemples de prétraitement:
Original: Saint-Nazaire se rêve en capitale des #énergies marines renouvelables #electricite http://t.co/nArcLYqxLd
Prétraité: saint nazaire se rêve en capitale des énergies marines renouvelables electricite http t co narclyqxld

Original: 4eme Conférence internationale sur le changement climatique et le développement http://t.co/biVoGZVlLj
Prétraité: 4eme conférence internationale sur le changement climatique et le développement http t co bivogzvllj

Original: Rencontres #windustry 2014 Sascha Wiesner décrit les innovations d'A2SEA dans le transport des éoliennes offshore http://t.co/hPD0DQPay6
Prétraité: rencontres windustry 2014 sascha wiesner décrit les innovations d a2sea dans le transport des éoliennes offshore http t co hpd0dqpay6



In [8]:
# Initialisation des compteurs pour chaque catégorie
positive_words = Counter()
negative_words = Counter()
neutral_words = Counter()

# Remplissage des compteurs
for _, row in train_df.iterrows():
    words = row['processed_text'].split()
    
    if row['label'] == '+':
        positive_words.update(words)
    elif row['label'] == '-':
        negative_words.update(words)
    else:
        neutral_words.update(words)

# Affichage des mots les plus fréquents pour chaque catégorie
print("Top 10 mots pour les tweets positifs:")
print(positive_words.most_common(10))

print("\nTop 10 mots pour les tweets négatifs:")
print(negative_words.most_common(10))

print("\nTop 10 mots pour les tweets neutres:")
print(neutral_words.most_common(10))

Top 10 mots pour les tweets positifs:
[('t', 1814), ('co', 1810), ('http', 1785), ('de', 1261), ('l', 1027), ('la', 840), ('le', 791), ('les', 731), ('des', 555), ('et', 510)]

Top 10 mots pour les tweets négatifs:
[('t', 1242), ('co', 1228), ('http', 1210), ('de', 1052), ('l', 773), ('la', 688), ('le', 649), ('les', 591), ('des', 507), ('et', 413)]

Top 10 mots pour les tweets neutres:
[('t', 2704), ('co', 2690), ('http', 2655), ('de', 2006), ('l', 1167), ('la', 1076), ('le', 1009), ('des', 803), ('et', 759), ('les', 756)]


In [9]:
# Filtrer les mots très fréquents/communs qui n'apportent pas d'information
# (articles, prépositions, etc.)
common_words = {'le', 'la', 'les', 'des', 'et', 'en', 'du', 'de', 'un', 'une', 
                'à', 'au', 'aux', 'pour', 'dans', 'sur', 'par', 'avec', 'ce', 
                'cette', 'ces', 'est', 'sont', 'ont', 'qui', 'que', 'quoi', 
                'comment', 'pourquoi', 'où', 'quand', 'pas', 'plus', 'moins'}

# Fonction pour filtrer les mots communs
def filter_counter(counter, common_words):
    return Counter({word: count for word, count in counter.items() 
                  if word not in common_words and len(word) > 1})

# Application du filtre
filtered_positive = filter_counter(positive_words, common_words)
filtered_negative = filter_counter(negative_words, common_words)
filtered_neutral = filter_counter(neutral_words, common_words)

# Affichage des résultats filtrés
print("Top 10 mots positifs (filtrés):")
print(filtered_positive.most_common(10))

print("\nTop 10 mots négatifs (filtrés):")
print(filtered_negative.most_common(10))

print("\nTop 10 mots neutres (filtrés):")
print(filtered_neutral.most_common(10))

Top 10 mots positifs (filtrés):
[('co', 1810), ('http', 1785), ('énergie', 289), ('climatique', 195), ('écosystème', 194), ('solaire', 189), ('via', 181), ('renouvelables', 176), ('écologie', 174), ('énergies', 170)]

Top 10 mots négatifs (filtrés):
[('co', 1228), ('http', 1210), ('climatique', 210), ('écologie', 197), ('via', 189), ('éoliennes', 143), ('ecologie', 142), ('réchauffement', 140), ('écologistes', 136), ('on', 129)]

Top 10 mots neutres (filtrés):
[('co', 2690), ('http', 2655), ('ecologie', 320), ('climatique', 311), ('via', 293), ('énergie', 271), ('durable', 263), ('développement', 259), ('écologie', 220), ('renouvelables', 218)]


In [10]:
# Fonction de classification par vocabulaire
def classify_by_vocabulary(text, positive_vocab, negative_vocab, neutral_vocab):
    # Prétraitement du texte
    processed = preprocess_text(text)
    words = processed.split()
    
    # Compter les mots dans chaque catégorie
    pos_score = sum(1 for word in words if word in positive_vocab)
    neg_score = sum(1 for word in words if word in negative_vocab)
    neu_score = sum(1 for word in words if word in neutral_vocab)
    
    # Déterminer la catégorie avec le plus grand score
    scores = {'+': pos_score, '-': neg_score, '=': neu_score}
    max_score = max(scores.values())
    
    # Si aucun mot du vocabulaire n'est trouvé
    if max_score == 0:
        return '='  # Par défaut: neutre
    
    # En cas d'égalité, ordonner par priorité
    if scores['='] == max_score:
        return '='
    elif scores['+'] == max_score:
        return '+'
    else:
        return '-'

# Convertir les compteurs en ensembles de vocabulaire (pour une recherche plus rapide)
pos_vocab = set(word for word, _ in filtered_positive.most_common(100))
neg_vocab = set(word for word, _ in filtered_negative.most_common(100))
neu_vocab = set(word for word, _ in filtered_neutral.most_common(100))

# Testons sur quelques exemples
test_examples = [
    train_df[train_df['label'] == '+']['text'].iloc[0],  # un exemple positif
    train_df[train_df['label'] == '-']['text'].iloc[0],  # un exemple négatif
    train_df[train_df['label'] == '=']['text'].iloc[0],  # un exemple neutre
]

print("Tests de classification:")
for example in test_examples:
    predicted = classify_by_vocabulary(example, pos_vocab, neg_vocab, neu_vocab)
    print(f"Texte: {example}")
    print(f"Prédiction: {predicted}")
    print()

Tests de classification:
Texte: Parc #éolien: entente conclut entre Port-Cartier la compagnie Système d'énergie renouvelable Canada: http://t.co/G9DWR1T792 #CoteNord
Prédiction: +

Texte: #Photos :Dans l’Ouest américain,les stigmates d’une sécheresse historique http://t.co/F6As9walnX #Environnement #Climat #EtatsUnis #Ecologie
Prédiction: =

Texte: Saint-Nazaire se rêve en capitale des #énergies marines renouvelables #electricite http://t.co/nArcLYqxLd
Prédiction: =



In [11]:
# Chargement des données de développement
dev_data = load_data('../data/dev.txt')
dev_df = pd.DataFrame(dev_data, columns=['text', 'label'])

# Application de la classification
predictions = []
for text in dev_df['text']:
    pred = classify_by_vocabulary(text, pos_vocab, neg_vocab, neu_vocab)
    predictions.append(pred)

# Calcul de l'exactitude (accuracy)
correct = sum(1 for pred, true in zip(predictions, dev_df['label']) if pred == true)
accuracy = correct / len(dev_df)

print(f"Exactitude sur l'ensemble de développement: {accuracy:.4f}")
print(f"Nombre de prédictions correctes: {correct}/{len(dev_df)}")

Exactitude sur l'ensemble de développement: 0.5657
Nombre de prédictions correctes: 396/700


In [12]:
# Version améliorée avec des poids
def classify_weighted(text, pos_counter, neg_counter, neu_counter):
    # Prétraitement
    processed = preprocess_text(text)
    words = processed.split()
    
    # Calcul des scores pondérés par la fréquence
    pos_score = sum(pos_counter.get(word, 0) for word in words)
    neg_score = sum(neg_counter.get(word, 0) for word in words)
    neu_score = sum(neu_counter.get(word, 0) for word in words)
    
    # Normalisation par le total de mots dans chaque catégorie
    total_pos = sum(pos_counter.values())
    total_neg = sum(neg_counter.values())
    total_neu = sum(neu_counter.values())
    
    if total_pos > 0: pos_score /= total_pos
    if total_neg > 0: neg_score /= total_neg
    if total_neu > 0: neu_score /= total_neu
    
    # Détermination de la catégorie
    scores = {'+': pos_score, '-': neg_score, '=': neu_score}
    
    if all(score == 0 for score in scores.values()):
        return '='  # Par défaut
    
    return max(scores, key=scores.get)

# Test de la version améliorée
weighted_predictions = []
for text in dev_df['text']:
    pred = classify_weighted(text, filtered_positive, filtered_negative, filtered_neutral)
    weighted_predictions.append(pred)

weighted_correct = sum(1 for pred, true in zip(weighted_predictions, dev_df['label']) if pred == true)
weighted_accuracy = weighted_correct / len(dev_df)

print(f"Exactitude avec pondération: {weighted_accuracy:.4f}")
print(f"Nombre de prédictions correctes: {weighted_correct}/{len(dev_df)}")

Exactitude avec pondération: 0.4900
Nombre de prédictions correctes: 343/700


In [13]:
# Chargement des données de test
test_data = load_data('../data/test.txt')
test_df = pd.DataFrame(test_data, columns=['text', 'label'])  # label sera "??"

# Génération des prédictions (utiliser la meilleure méthode entre les deux)
if weighted_accuracy > accuracy:
    print("Utilisation de la méthode pondérée pour les prédictions finales")
    test_predictions = []
    for text in test_df['text']:
        pred = classify_weighted(text, filtered_positive, filtered_negative, filtered_neutral)
        test_predictions.append(pred)
else:
    print("Utilisation de la méthode simple pour les prédictions finales")
    test_predictions = []
    for text in test_df['text']:
        pred = classify_by_vocabulary(text, pos_vocab, neg_vocab, neu_vocab)
        test_predictions.append(pred)

# Création du fichier de sortie
with open('../results/test-predict.txt', 'w', encoding='utf-8') as f:
    for (text, _), prediction in zip(test_data, test_predictions):
        f.write(f"{text}\t{prediction}\n")

print("Prédictions générées et enregistrées dans 'test-predict.txt'")

Utilisation de la méthode simple pour les prédictions finales
Prédictions générées et enregistrées dans 'test-predict.txt'


In [14]:
# Structure finale pour le script Python
def main_function():
    # Chargement et préparation des données d'entraînement
    train_data = load_data('train.txt')
    train_df = pd.DataFrame(train_data, columns=['text', 'label'])
    train_df['processed_text'] = train_df['text'].apply(preprocess_text)
    
    # Construction des vocabulaires
    positive_words = Counter()
    negative_words = Counter()
    neutral_words = Counter()
    
    for _, row in train_df.iterrows():
        words = row['processed_text'].split()
        if row['label'] == '+':
            positive_words.update(words)
        elif row['label'] == '-':
            negative_words.update(words)
        else:  # label == '='
            neutral_words.update(words)
    
    # Filtrage des mots communs
    common_words = {'le', 'la', 'les', 'des', 'et', 'en', 'du', 'de', 'un', 'une', 
                    'à', 'au', 'aux', 'pour', 'dans', 'sur', 'par', 'avec', 'ce', 
                    'cette', 'ces', 'est', 'sont', 'ont', 'qui', 'que', 'quoi', 
                    'comment', 'pourquoi', 'où', 'quand', 'pas', 'plus', 'moins'}
    
    filtered_positive = filter_counter(positive_words, common_words)
    filtered_negative = filter_counter(negative_words, common_words)
    filtered_neutral = filter_counter(neutral_words, common_words)
    
    # Création des vocabulaires pour la classification
    pos_vocab = set(word for word, _ in filtered_positive.most_common(100))
    neg_vocab = set(word for word, _ in filtered_negative.most_common(100))
    neu_vocab = set(word for word, _ in filtered_neutral.most_common(100))
    
    # Lecture du fichier d'entrée et génération des prédictions
    # (à adapter selon les arguments de ligne de commande)
    input_file = 'dev.txt'  # Sera remplacé par sys.argv[1]
    
    input_data = load_data(input_file)
    predictions = []
    
    for text, _ in input_data:
        pred = classify_by_vocabulary(text, pos_vocab, neg_vocab, neu_vocab)
        predictions.append(pred)
    
    # Écriture des résultats
    for (text, _), prediction in zip(input_data, predictions):
        print(f"{text}\t{prediction}")

